<a href="https://colab.research.google.com/github/2018007956/HYU/blob/main/Deep_Learning/11_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. RNN을 이용한 character prediction

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)

if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')

In [2]:
sentence = ' seoul is the capital of south korea'
char_set = list(set(sentence))
char_dic = {c:i for i,c in enumerate(char_set)}

In [3]:
char_set

[' ', 'l', 'p', 'a', 'u', 't', 'r', 'i', 'e', 'k', 'o', 's', 'h', 'f', 'c']

In [4]:
char_dic

{' ': 0,
 'l': 1,
 'p': 2,
 'a': 3,
 'u': 4,
 't': 5,
 'r': 6,
 'i': 7,
 'e': 8,
 'k': 9,
 'o': 10,
 's': 11,
 'h': 12,
 'f': 13,
 'c': 14}

In [5]:
vocab_sz = len(char_dic)
hidden_sz = len(char_dic)
input_sz = len(char_dic)

In [6]:
'''
Sentence의 character index로 변경
input:" seoul is the capital of south kore"의 index input을 one hot vector로 변경
target:"seoul is the capital of south korea:
'''
sen_idx = [char_dic[c] for c in sentence]
x_idx = sen_idx[:-1]
x_one_hot = [[np.eye(vocab_sz)[x] for x in x_idx]]
y_data = [sen_idx[1:]]

In [7]:
print(sen_idx)
print(x_idx)
print(x_one_hot)
print(y_data)

[0, 11, 8, 10, 4, 1, 0, 7, 11, 0, 5, 12, 8, 0, 14, 3, 2, 7, 5, 3, 1, 0, 10, 13, 0, 11, 10, 4, 5, 12, 0, 9, 10, 6, 8, 3]
[0, 11, 8, 10, 4, 1, 0, 7, 11, 0, 5, 12, 8, 0, 14, 3, 2, 7, 5, 3, 1, 0, 10, 13, 0, 11, 10, 4, 5, 12, 0, 9, 10, 6, 8]
[[array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]), array([0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]), array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]), array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), ar

In [8]:
x_train = torch.FloatTensor(x_one_hot)
y_train = torch.LongTensor(y_data)

<ipython-input-8-9278ed7c0bad>:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  x_train = torch.FloatTensor(x_one_hot)


In [9]:
class Rnn(nn.Module):
  def __init__(self, input_size, hidden_size, vocab_size):
    super(Rnn, self).__init__()
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.vocab_size = vocab_size

    self.rnn = nn.RNN(input_size=self.input_size, hidden_size=self.hidden_size, batch_first=True)
    self.linear = nn.Linear(self.hidden_size, self.vocab_size)

  def forward(self, x):
    outputs, _ = self.rnn(x)
    x = self.linear(outputs)

    return x

model = Rnn(input_size = input_sz, hidden_size=hidden_sz, vocab_size=vocab_sz).to(device)

In [10]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [11]:
epochs = 500

for epoch in range(epochs):
  model.train()

  outputs = model(x_train.to(device)) # forward propagation
  loss = criterion(outputs.view(-1, hidden_sz), y_train.view(-1).to(device))

  optimizer.zero_grad()
  loss.backward() # backward propagation
  optimizer.step() # update parameters

  result = outputs.data.cpu().numpy().argmax(axis=2)
  result_str = ''.join([char_set[idx] for idx in np.squeeze(result)])

  if epoch % 50 == 0 or epoch == epochs-1:
    print('loss : {} prediction : {}'.format(loss, result_str))

loss : 2.820239782333374 prediction : uukuuuuuuuuukuuuuuuuuuuuuusuuuuuuuu
loss : 2.5405964851379395 prediction :                                    
loss : 2.3091976642608643 prediction : s      s     so               sso  
loss : 1.9457604885101318 prediction : sooil  s     sroit      sout  sorea
loss : 1.5533915758132935 prediction : sooil  s  h  capithl su soul  korea
loss : 1.1812185049057007 prediction : sooul ss  he capital ou soul  korea
loss : 0.860175371170044 prediction : seoul is the capital of south korea
loss : 0.6095166206359863 prediction : seoul is the capital of south korea
loss : 0.43181368708610535 prediction : seoul is the capital of south korea
loss : 0.3119470477104187 prediction : seoul is the capital of south korea
loss : 0.2333676517009735 prediction : seoul is the capital of south korea


## 2. LSTM